In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Allergies/GSE182740'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genomic profiling of the overlap phenotype between psoriasis and atopic dermatitis"
!Series_summary	"Clinical overlaps between psoriasis and atopic dermatitis are sometimes undiscernible, and there is no consensus whether to treat the overlap phenotype as psoriasis or atopic dermatitis. We enrolled patients diagnosed with either psoriasis or atopic dermatitis, and clinically re-stratified them into classic psoriasis, classic atopic dermatitis, and the overlap phenotype between psoriasis and atopic dermatitis. We compared gene expression profiles of lesional and nonlesional skin biopsy tissues between the three comparison groups. Global mRNA expression and T-cell subset cytokine expression in the skin of the overlap phenotype were consistent with the profiles of psoriasis and different from the profiles of atopic dermatitis. Unsupervised k-means clustering indicated that the best number of distinct clusters for the total population of the three com

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

import os
import pandas as pd  # Assuming pandas import is necessary for further processing
import json

def extract_value(cell):
    if pd.isna(cell):
        return None
    return cell.split(': ')[1] if isinstance(cell, str) and ': ' in cell else None

# Check if the dataset is likely to contain gene expression data
if 'gene expression' in "!Series_summary".lower():
    is_gene_available = True

# Define explicit conversion functions that return None
def convert_trait(x):
    return None

def convert_age(x):
    return None

def convert_gender(x):
    return None

# Save cohort information
def save_cohort_info(series_id, filepath, is_gene_available, is_clinical_available):
    cohort_info = {
        "series_id": series_id, 
        "is_gene_available": is_gene_available, 
        "is_clinical_available": is_clinical_available
    }
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w') as f:
        json.dump(cohort_info, f)

save_cohort_info('GSE182740', './preprocessed/Allergies/cohort_info.json', is_gene_available, trait_row is not None)

# If trait_row is available, then extract clinical features
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Allergies', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Allergies/trait_data/GSE182740.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
